In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
from datetime import datetime, timedelta
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store
from src.data_utils import transform_ts_data_info_features
from src.inference import load_batch_of_features_from_store
from src.inference import load_model_from_registry
from src.inference import get_model_predictions

c:\Users\singh\Downloads\CDS500_Applied_ML_DS\Projects\CDA500P1\CDA500P1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data.info()

2025-03-04 21:20:04,871 INFO: Initializing external client
2025-03-04 21:20:04,872 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:20:05,766 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Fetching data from 2025-02-04 02:20:04.869119+00:00 to 2025-03-05 01:20:04.869119+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.36s) 


,pickup_hour,pickup_location_id,rides
0,2025-02-04 03:00:00+00:00,2,0
1,2025-02-04 04:00:00+00:00,2,0
2,2025-02-04 05:00:00+00:00,2,0
3,2025-02-04 06:00:00+00:00,2,0
4,2025-02-04 07:00:00+00:00,2,0
...,...,...,...
174440,2025-03-04 21:00:00+00:00,263,76
174441,2025-03-04 22:00:00+00:00,263,58
174442,2025-03-04 23:00:00+00:00,263,30
174443,2025-03-05 00:00:00+00:00,263,22


<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 175699
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [3]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)
ts_data.info()
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)
features.head()
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)
current_date
features.head()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 175699
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,3,0,3,5,1,21,0,0,...,0,1,2,1,1,0,14,0,255,2025-02-16 01:00:00
1,0,0,3,7,0,5,3,10,9,8,...,8,18,3,15,0,22,14,2,209,2025-02-25 00:00:00
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,56,2025-02-27 02:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,171,2025-02-19 23:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,2025-02-26 07:00:00


2025-03-04 21:20:23,561 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:20:23,567 INFO: Initializing external client
2025-03-04 21:20:23,567 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:20:24,277 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Fetching data from 2025-02-04 02:20:23.561312+00:00 to 2025-03-05 01:20:23.561312+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.28s) 


Timestamp('2025-03-05 02:20:23.561312+0000', tz='Etc/UTC')

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 03:00:00
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 03:00:00
2,0,1,1,2,1,11,7,3,2,3,...,0,0,1,1,1,4,4,3,4,2025-03-04 03:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 03:00:00
4,0,0,1,2,2,1,2,3,2,3,...,5,2,1,1,0,0,1,0,7,2025-03-04 03:00:00


In [4]:
model = load_model_from_registry()

2025-03-04 21:20:45,677 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:20:45,683 INFO: Initializing external client
2025-03-04 21:20:45,686 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:20:46,462 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689


In [5]:
predictions = get_model_predictions(model, features)
predictions
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,0.0
...,...,...
246,259,0.0
247,260,0.0
248,261,0.0
249,262,2.0


array([ 48, 132, 230,  79, 186,  68, 100, 249, 107,  90])